In [235]:
import pandas as pd
import numpy as np
import optuna
import plotly

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_pinball_loss, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
from sklearn.model_selection import KFold


In [ ]:
# More informations about this dataset here :https://vitaldb.net/dataset/?query=overview#h.1fo5zknztqnw
df = pd.read_csv("operation_length_predictions_vitalDB.csv") 

### Preprocessing

In [244]:
import pandas as pd
def calculate_duration_cols(df):
    """Calculate time durations for hospital, operation and anesthesia in hours from seconds"""
    df['duration_stay'] = (df['dis'] - df['adm'])/3600
    df['duration_operation'] = (df['opend'] - df['opstart'])/3600
    df['duration_anesthesia'] = (df['aneend'] - df['anestart'])/3600
    return df

def drop_id_cols(df):
    """Drop ID columns and original timestamp columns"""
    columns_to_drop = [
        'caseid', 'subjectid',
        'dis', 'adm',
        'opend', 'opstart',
        'aneend', 'anestart',
        'casestart','caseend', 
        'icu_days',"dx","opname",
        'preop_ecg','preop_ph', 'preop_hco3', 'preop_be', 'preop_pao2', 
        'preop_paco2', 'preop_sao2', 'cormack', 'tubesize',
        'dltubesize', 'lmasize', 'iv2', 'aline1', 'aline2',
        'cline1', 'cline2',"preop_na","preop_k",'airway','iv1','duration_stay',
        'death_inhosp'
    ]

    df = df.drop(columns=df.filter(like='intraop').columns)

    return df.drop(columns_to_drop, axis=1)

def map_position(df):
    position_mapping = {
    'Supine': 'Supine',
    'Lithotomy': 'Lithotomy',
    'Left lateral decubitus': 'Lateral Decubitus',
    'Right lateral decubitus': 'Lateral Decubitus',
    'Prone': 'Prone',
    'Reverse Trendelenburg': 'Trendelenburg (Inclined)',
    'Trendelenburg': 'Trendelenburg (Inclined)',
    'Sitting': 'Sitting',
    'Left kidney': 'Lateral Decubitus',
    'Right kidney': 'Lateral Decubitus'
    }

    df['position'] = df['position'].map(position_mapping)
    return df

def map_ane_type(df):
    ane_mapping = {
        'General': 'General Anesthesia',
        'Spinal': 'Regional Anesthesia',
        'Sedationalgesia': 'Regional Anesthesia'
    }

    df['ane_type'] = df['ane_type'].map(ane_mapping)
    return df

def handle_age_more_than_89(df):
    df.loc[df['age'] == '>89', 'age'] = 92
    return df

def map_preop_pft(df):
    pft_mapping = {
    "Normal": "Normal",
    "Mild obstructive": "Mild/Moderate Obstructive",
    "Moderate obstructive": "Mild/Moderate Obstructive",
    "Borderline obstructive": "Mild/Moderate Obstructive",
    "Severe obstructive": "Severe Obstructive",
    "Mixed or pure obstructive": "Severe Obstructive",
    "Mild restrictive": "Mild/Moderate Restrictive",
    "Moderate restrictive": "Mild/Moderate Restrictive",
    "Severe restrictive": "Severe Restrictive",
    }
    df['preop_pft'] = df['preop_pft'].map(pft_mapping)
    return df
    

def drop_na_rows(df):
    return df.dropna().reset_index(drop=True)

def encode_scale(df):

    # label_encoder = OrdinalEncoder()
    scaler = StandardScaler()
    onehot_encoder = OneHotEncoder(handle_unknown='ignore')

    # categorical_features = []
    numerical_features = ['age', 'weight','height', 'bmi', 'asa', 'preop_hb', 'preop_plt', 'preop_pt', 'preop_aptt', 'preop_gluc', 'preop_alb', 'preop_ast', 'preop_alt', 'preop_bun','preop_cr']
    onehot_features = ['preop_pft','sex', 'department', 'optype', 'approach', 'ane_type', 'position']

    preprocessor = ColumnTransformer(
    transformers=[
        # ('cat', label_encoder, categorical_features),
        ('num', scaler, numerical_features),
        ('onehot', onehot_encoder, onehot_features)
    ],
    remainder='passthrough',
    # verbose_feature_names_out=False
    )
    df_processed = preprocessor.fit_transform(df)
    return df_processed

def split_dataset(df,target_column='duration_anesthesia'):
    """Split the dataset into train and test sets"""
    y = df[target_column]
    X = df.drop(columns=['duration_operation','duration_anesthesia'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def handle_outliers(X_train,y_train,target_column):
    """Handle outliers in the training set"""
    df = pd.concat([X_train, y_train], axis=1)
    df = df.drop(df[df[target_column] > 10].index)
    df = df.drop(df[df[target_column] < 0].index)

    y_train = df[target_column]
    X_train = df.drop(columns=[target_column], axis=1)
    print(X_train.shape)
    return X_train,y_train

def process_data(df,target_column):
    """Main function to process the dataframe"""
    df = calculate_duration_cols(df)
    df = drop_id_cols(df)
    df = drop_na_rows(df)
    df = map_position(df)
    df = map_ane_type(df)
    df = handle_age_more_than_89(df)
    df = map_preop_pft(df)
    # print(df.columns)
    X_train, X_test, y_train, y_test = split_dataset(df,target_column)
    X_train, y_train = handle_outliers(X_train,y_train,target_column)
    X_train = encode_scale(X_train)
    X_test = encode_scale(X_test)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = process_data(df, target_column='duration_operation')

(4526, 25)


### Tuning with optuna

In [245]:
def objective(trial, X, y, n_splits=4):
    """Optuna objective function with cross-validation for 0.95 quantile"""
    # Define hyperparameter search space
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 6),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0)
    }
    
    # Initialize cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    
    # Perform cross-validation
    for train_idx, val_idx in kf.split(X):
        X_fold_train, X_fold_val = X[train_idx], X[val_idx]
        y_fold_train, y_fold_val = y[train_idx], y[val_idx]
        
        # Train model for 95th percentile
        model = GradientBoostingRegressor(
            loss="quantile",
            alpha=0.95,
            random_state=42,
            **params
        )
        
        # Fit and evaluate
        model.fit(X_fold_train, y_fold_train)
        y_pred = model.predict(X_fold_val)
        
        # Calculate pinball loss
        score = mean_pinball_loss(y_fold_val, y_pred, alpha=0.95)
        scores.append(score)
    
    return np.mean(scores)

def optimize_and_train_model(X_train, y_train, n_trials=10):
    """Optimize hyperparameters and return the best model"""
    # Create study
    study = optuna.create_study(direction="minimize")
    
    # Optimize
    study.optimize(lambda trial: objective(trial, X_train, y_train), 
                  n_trials=n_trials,
                  show_progress_bar=True)
    
    # Get best parameters
    best_params = study.best_params
    print("Best parameters:", best_params)
    
    # Train final model with best parameters
    final_model = GradientBoostingRegressor(
        loss="quantile",
        alpha=0.95,
        random_state=42,
        **best_params
    )
    final_model.fit(X_train, y_train)
    
    return final_model, study,best_params

def evaluate_model(model, X_test, y_test):
    """Evaluate the model on test data"""
    y_pred = model.predict(X_test)
    pinball_loss = mean_pinball_loss(y_test, y_pred, alpha=0.95)
    
    # Calculate percentage of predictions above actual values
    coverage = np.mean(y_test <= y_pred)
    
    return {
        'pinball_loss': pinball_loss,
        'coverage': coverage
    }


X_train_array = np.array(X_train)
y_train_array = np.array(y_train)

best_model, study,best_params = optimize_and_train_model(
    X_train_array, 
    y_train_array, 
    n_trials=10
)

metrics = evaluate_model(best_model, X_test, y_test)
print("\nModel Evaluation:")
print(f"Pinball Loss: {metrics['pinball_loss']:.4f}")
print(f"Coverage (should be close to 0.95): {metrics['coverage']:.4f}")

[I 2025-02-15 18:34:54,771] A new study created in memory with name: no-name-deceb4eb-d87c-400e-bd82-745d2bd548ff
Best trial: 0. Best value: 0.18862:  10%|█         | 1/10 [01:17<11:35, 77.23s/it]

[I 2025-02-15 18:36:11,992] Trial 0 finished with value: 0.18862045292599353 and parameters: {'learning_rate': 0.0823682974050483, 'n_estimators': 428, 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 14, 'subsample': 0.615516319094151}. Best is trial 0 with value: 0.18862045292599353.


Best trial: 1. Best value: 0.179536:  20%|██        | 2/10 [01:41<06:10, 46.29s/it]

[I 2025-02-15 18:36:36,622] Trial 1 finished with value: 0.17953565462037208 and parameters: {'learning_rate': 0.1687459998414928, 'n_estimators': 185, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'subsample': 0.8917530900976749}. Best is trial 1 with value: 0.17953565462037208.


Best trial: 2. Best value: 0.174927:  30%|███       | 3/10 [02:24<05:12, 44.63s/it]

...

[I 2025-02-15 18:41:06,214] Trial 7 finished with value: 0.18495531349051617 and parameters: {'learning_rate': 0.020157808036741363, 'n_estimators': 193, 'max_depth': 2, 'min_samples_leaf': 14, 'min_samples_split': 11, 'subsample': 0.9109759717399742}. Best is trial 2 with value: 0.17492738627297155.


Best trial: 2. Best value: 0.174927:  90%|█████████ | 9/10 [07:33<00:56, 56.27s/it]

[I 2025-02-15 18:42:28,520] Trial 8 finished with value: 0.17736639541628485 and parameters: {'learning_rate': 0.031409907348925487, 'n_estimators': 412, 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 5, 'subsample': 0.6718053540612333}. Best is trial 2 with value: 0.17492738627297155.


Best trial: 9. Best value: 0.174355: 100%|██████████| 10/10 [07:59<00:00, 47.94s/it]


[I 2025-02-15 18:42:54,155] Trial 9 finished with value: 0.17435500774868676 and parameters: {'learning_rate': 0.2549625640188951, 'n_estimators': 260, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 13, 'subsample': 0.882623021280744}. Best is trial 9 with value: 0.17435500774868676.
Best parameters: {'learning_rate': 0.2549625640188951, 'n_estimators': 260, 'max_depth': 2, 'min_samples_leaf': 3, 'min_samples_split': 13, 'subsample': 0.882623021280744}

Model Evaluation:
Pinball Loss: 0.1640
Coverage (should be close to 0.95): 0.9339


### Train  upper bound model

In [ ]:
def train_quantile_models(X_train, y_train):
    """Train Gradient Boosting models for quantile regression with alpha=0.5 and alpha=0.95 (upper bound with 95% confidence)"""
    all_models = {}
    common_params = dict(
        learning_rate=0.05,
        n_estimators=200,
        max_depth=2,
        min_samples_leaf=9,
        min_samples_split=9,
    )

    for alpha in [0.5, 0.95]:
        model = GradientBoostingRegressor(loss="quantile", alpha=alpha, **common_params)
        all_models[f"q {alpha:.2f}"] = model.fit(X_train, y_train)
    return all_models

def evaluate_models(all_models, X_test, y_test):
    """Compute evaluation metrics for quantile regression models"""
    y_pred_dict = {q: model.predict(X_test) for q, model in all_models.items()}

    y_pred_median = y_pred_dict["q 0.50"]
    y_pred_upper = y_pred_dict["q 0.95"]

    # Compute metrics
    pinball_losses = {q: mean_pinball_loss(y_test, y_pred_dict[q], alpha=float(q.split()[1])) 
                      for q in y_pred_dict}

    coverage = (y_test <= y_pred_upper).mean()  # Since we have only the upper bound
    mse_median = mean_squared_error(y_test, y_pred_median)
    mae_median = mean_absolute_error(y_test, y_pred_median)

    results = {
        "Pinball Losses": pinball_losses,
        "Coverage (95%)": coverage,
        "MSE (Median Model)": mse_median,
        "MAE (Median Model)": mae_median
    }

    return results

# Example usage:
all_models = train_quantile_models(X_train, y_train)
metrics = evaluate_models(all_models, X_test, y_test)

for key, value in metrics.items():
    print(f"{key}: {value}")


Pinball Losses: {'q 0.50': 0.41926749970344107, 'q 0.95': 0.17209318888750916}
Coverage (95%): 0.9444444444444444
MSE (Median Model): 1.6425067568037361
MAE (Median Model): 0.8385349994068821


### Train on all datas

In [246]:
def train_upper_quantile_model(X_full, y_full):
    """Train a single Gradient Boosting model for 0.95 quantile"""
    model = GradientBoostingRegressor(
        loss="quantile",
        alpha=0.95,
        **best_params # best params identified during the tuning procedure
    )
    model.fit(X_full, y_full)
    return model

# Combine training and test data
X_full = np.concatenate([X_train, X_test], axis=0)
y_full = np.concatenate([y_train, y_test], axis=0)

# Train model
upper_model = train_upper_quantile_model(X_full, y_full)

### Save final model

In [ ]:
# Save model
import joblib
joblib.dump(upper_model, 'quantile_model_95.joblib')

# To load and use the model later:
# loaded_model = joblib.load('quantile_model_95.joblib')
# predictions = loaded_model.predict(X_new)

['quantile_model_95.joblib']